In [ ]:
import os 
os.chdir(r'Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp\src')

from neurochat.nc_data import NData
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
import neurochat.nc_plot as nc_plot
from neurochat.nc_lfp import NLfp
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np
import math
import pynapple as nap
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

import sys
import nwb4fp.analyses.maps as mapp
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap,plot_path
from nwb4fp.analyses.fields import separate_fields_by_laplace, separate_fields_by_dilation,find_peaks,separate_fields_by_laplace_of_gaussian,calculate_field_centers,distance_to_edge_function, remove_fields_by_area, map_pass_to_unit_circle,which_field,compute_crossings
from elephant.statistics import time_histogram, instantaneous_rate
from nwb4fp.analyses import maps
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,unit_location_ch

from scipy.ndimage import gaussian_filter
import ast
import pandas as pd
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

In [ ]:
import numpy as np

import numpy as np

def calculate_burst_index(spike_times, bin_size=0.001, max_lag=0.1):
    """
    Calculate the burst index from spike times (in seconds) using the auto-correlogram.
    
    Parameters:
    spike_times : array-like
        List or array of spike times in seconds.
    bin_size : float, optional
        Bin size for the auto-correlogram in seconds (default: 0.001 s = 1 ms).
    max_lag : float, optional
        Maximum lag for the auto-correlogram in seconds (default: 0.05 s = 50 ms).
    
    Returns:
    float
        Burst index ranging from -1 to 1. Returns np.nan if insufficient data.
    """
    # Input validation
    spike_times = np.array(spike_times, dtype=float)
    if len(spike_times) < 2:
        raise ValueError("At least two spike times are required.")
    if max_lag <= 0 or bin_size <= 0:
        raise ValueError("max_lag and bin_size must be positive.")

    # Sort spike times for efficiency
    spike_times = np.sort(spike_times)
    
    # Compute pairwise differences efficiently
    diffs = []
    for i, t in enumerate(spike_times):
        future_spikes = spike_times[i+1:]
        valid_diffs = future_spikes - t
        valid_diffs = valid_diffs[valid_diffs <= max_lag]
        if len(valid_diffs) > 0:
            diffs.extend(valid_diffs)
            #diffs.extend(-valid_diffs)  # Include negative differences for symmetry

    diffs = np.array(diffs)
    
    # Create bins
    bins = np.arange(-max_lag, max_lag + bin_size, bin_size)
    
    # Compute histogram
    hist, bin_edges = np.histogram(diffs, bins=bins)
    
    # Normalize to probability density (spikes per second)
    total_spikes = len(spike_times)
    autocorr = hist / (np.sum(hist))

    #autocorr = hist / total_pairs if total_pairs > 0 else hist
    
    # Calculate indices for baseline (40-50 ms = 0.04-0.05 s) and peak (0-10 ms = 0-0.01 s)
    baseline_idx = np.where((bins[:-1] >= 0.05) & (bins[:-1] < 0.1))[0]
    peak_idx = np.where((bins[:-1] >= 0) & (bins[:-1] <= 0.006))[0]
    
    # Check if indices are valid
    if len(baseline_idx) == 0 or len(peak_idx) == 0:
        return np.nan
    
    # Calculate baseline (mean value between 40 and 50 ms)
    baseline = np.mean(autocorr[baseline_idx])
    
    # Calculate peak (max value between 0 and 10 ms)
    peak = np.max(autocorr[peak_idx]) if len(peak_idx) > 0 else 0
    
    # Calculate burst amplitude
    burst_amplitude = peak - baseline
    
    # Normalize burst amplitude
    if burst_amplitude > 0:
        # Normalize to peak for positive amplitudes
        burst_index = burst_amplitude / peak if peak != 0 else 0
    else:
        # Normalize to baseline for negative amplitudes
        burst_index = burst_amplitude / baseline if baseline != 0 else 0
    
    return hist, autocorr, burst_index

def calculate_mean_firing_rate(spike_times):
    """
    Calculate mean firing rate (spikes per second) from a list of spike times in seconds.
    
    Parameters:
    - spike_times: List or array of spike times in seconds
    
    Returns:
    - firing_rate: Mean firing rate in Hz (spikes/second)
    """
    if len(spike_times) < 1:
        return 0.0
    total_spikes = len(spike_times)
    time_span = max(spike_times) - min(spike_times) if total_spikes > 1 else 1.0  # Avoid division by zero
    return total_spikes / time_span

def compute_autocorrelogram(spike_times, bins, max_lag=0.1):
    spike_times = np.array(spike_times)
    autocorr = np.zeros(len(bins) - 1)
    
    # Compute differences between all pairs of spikes
    for i, t1 in enumerate(spike_times):
        diffs = t1 - spike_times[i+1:]  # Exclude self-correlation at lag 0
        valid_diffs = diffs[(diffs >= -max_lag) & (diffs <= max_lag)]
        hist, _ = np.histogram(valid_diffs, bins=bins)
        autocorr += hist
    
    return autocorr

import numpy as np

# Example spike times in seconds (replace this with your own data)

def mean_isi(spike_times):
    # Calculate inter-spike intervals (ISI) by subtracting consecutive times
    import numpy as np
    isi = np.diff(spike_times)
    # Calculate the mean ISI
    return np.mean(isi)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import multiprocessing
from functools import partial

def acg_narrow(spike_times):
    bin_size=0.0005
    max_lag=0.05
    spike_times = np.array(spike_times, dtype=float)
    if len(spike_times) < 2:
        raise ValueError("At least two spike times are required.")
    if max_lag <= 0 or bin_size <= 0:
        raise ValueError("max_lag and bin_size must be positive.")

    # Sort spike times for efficiency
    spike_times = np.sort(spike_times)

    # Compute pairwise differences efficiently
    diffs = []
    for i, t in enumerate(spike_times):
        future_spikes = spike_times[i+1:]
        valid_diffs = future_spikes - t
        valid_diffs = valid_diffs[valid_diffs <= max_lag]
        if len(valid_diffs) > 0:
            diffs.extend(valid_diffs)
            diffs.extend(-valid_diffs)  # Include negative differences for symmetry

    diffs = np.array(diffs)

    # Create bins
    bins = np.arange(-max_lag, max_lag + bin_size, bin_size)

    # Compute histogram
    hist, bin_edges = np.histogram(diffs, bins=bins) 

    return hist

import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

def triple_exp(x, a, b, c, d, e, f, g, h):
    return np.maximum(c * (np.exp(-(x - f) / a) - d * np.exp(-(x - f) / b)) + h * np.exp(-(x - f) / g) + e, 0)

def fit_ACG(acg_narrow, plots=True):
    acg_narrow = acg_narrow.copy()
    acg_narrow[99:101, :] = 0  # set time-zero bin to zero (-0.5ms -> 0.5ms)
    offset = 100
    x = np.arange(0, 100) / 2  # [0, 0.5, 1, ..., 49.5] ms

    # Initial guess, lower bounds, upper bounds
    p0 = [20, 1, 30, 2, 0.5, 5, 1.5, 2]
    lb = [1, 0.1, 0, 0, -30, 0, 0.1, 0]
    ub = [500, 50, 500, 15, 50, 20, 5, 100]
    bounds = (lb, ub)

    n_cells = acg_narrow.shape[1]
    fit_params = np.full((8, n_cells), np.nan)
    rsquare = np.full(n_cells, np.nan)
    first_positive_x = np.full(n_cells, np.nan)  # New array for first x where y > 0
    plotf0 = np.zeros((100, n_cells))

    for j in range(n_cells):
        if not np.any(np.isnan(acg_narrow[:, j])):
            y = acg_narrow[(x * 2 + offset).astype(int), j]
            try:
                popt, pcov = curve_fit(triple_exp, x, y, p0=p0, bounds=bounds, maxfev=10000)
                fit_params[:, j] = popt
                y_fit = triple_exp(x, *popt)
                plotf0[:, j] = y_fit
                # R-square calculation
                ss_res = np.sum((y - y_fit) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                rsquare[j] = 1 - (ss_res / ss_tot)

                # Find first x where y > 0 using high-resolution x_fit
                x_fit = np.arange(0, 100.1, 0.1) / 2  # [0, 0.05, 0.1, ..., 50] ms
                y_fit_hr = triple_exp(x_fit, *popt)
                positive_idx = np.where(y_fit_hr > 0)[0]
                if positive_idx.size > 0:
                    first_positive_x[j] = x_fit[positive_idx[0]]  # First x where y > 0

                if plots:
                    plt.figure()
                    plt.plot(x, y, 'b-', label='ACG')
                    plt.plot(x_fit, y_fit_hr, 'r--', label='Fit')
                    plt.title(f'Cell {j+1} Fit')
                    plt.xlabel('Time (ms)')
                    plt.ylabel('Counts')
                    plt.legend()
                    param_text = '\n'.join(f'{name}={val:.2f}' for name, val in zip(
                        ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], popt))
                    param_text += f'\nfirst_positive_x={first_positive_x[j]:.2f}'
                    plt.text(0.5, 0.5, param_text, transform=plt.gca().transAxes)
                    plt.show()

            except RuntimeError:
                print(f"Fit failed for cell {j}")

    fit_params_out = {
        'acg_tau_decay': fit_params[0, :],
        'acg_tau_rise': fit_params[1, :],
        'acg_c': fit_params[2, :],
        'acg_d': fit_params[3, :],
        'acg_asymptote': fit_params[4, :],
        'acg_refrac': fit_params[5, :],
        'acg_tau_burst': fit_params[6, :],
        'acg_h': fit_params[7, :],
        'acg_fit_rsquare': rsquare,
        'acg_first_positive_x': first_positive_x  # New output parameter
    }

    return fit_params_out

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import shapiro, levene, kruskal
import statsmodels.api as sm
from statsmodels.formula.api import ols
#import scikit_posthocs as sp

# Folder path
folder_path = r"S:\Sachuriga\file_with_table\ripple_ch/"

# Function to get pickle files
def get_pkl_files(folder_path):
    all_files = os.listdir(folder_path)
    pkl_files = [f for f in all_files if f.endswith("withDLC.pkl")]
    return pkl_files
pkl_files = get_pkl_files(folder_path)


# Define group prefixes
target_prefixes_control = ['65165', '65091', '63383', '66539', '65622']
target_prefixes_exp = ['65588', '63385', '66538', '66537', '66922']

# Get the list of pickle files
pkl_files = get_pkl_files(folder_path)

# Initialize a list to store all DataFrames
all_dfs = []

# Process each pickle file
for file in pkl_files:
    df = pd.read_pickle(fr"{folder_path}/{file}")
    df['buzaki_cell_type'] = None
    df_good = df
    for i in range(len(df_good)):
        if (df_good['peak_to_valley'].iloc[i] <= 0.000425)|(df_good['mean_firing_rate'].iloc[i] >= 10):
            df_good['buzaki_py_cell_type'].iloc[i] = "narrow_spike_interneurons"
        #elif  (df_good['peak_to_valley'].iloc[i] > 0.000425) & (((df_good['tau_rise_python'].iloc[i]) + (df_good['acg_first_positive_x'].iloc[i])) >= 6):
        elif  (df_good['peak_to_valley'].iloc[i] > 0.000425) & ((df_good['tau_rise_python'].iloc[i])  >= 6):
            df_good['buzaki_py_cell_type'].iloc[i] = "wide_spike_interneurons"
        #elif (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['mean_firing_rate'].iloc[i] < 10) & (((df_good['tau_rise_python'].iloc[i]) + (df_good['acg_first_positive_x'].iloc[i])) < 6):
        elif (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['mean_firing_rate'].iloc[i] < 10) & ((df_good['tau_rise_python'].iloc[i])  < 6):
            df_good['buzaki_py_cell_type'].iloc[i] = "pyramidal"
        
    all_dfs.append(df_good.reset_index(drop=True))

combines = pd.concat(all_dfs, ignore_index=False)
combines.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

In [ ]:
len(combines)

In [ ]:
import pandas as pd
from scipy.stats import mode
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
len(df_loaded)
df_good = df_loaded[df_loaded['unit_quality']=="good"]

df_good.head()
# df_good['matlab_acg_tau_rise']
# df_good['peak_to_valley']
# sns.scatterplot(x='peak_to_valley', y='matlab_acg_tau_rise', hue='cell_type', data=df_good)
# plt.ylabel('Tau Rise (ms)')
# plt.xlabel('Trough to Peak (ms)')
# plt.title('Scatter Plot Colored by Cell Type')
# plt.show()
bursting_index=[]
mean_firing_rate=[]
auto_corr=[]
df_good['buzaki_cell_type'] = None
df_good["bursting_index"] = None
df_good["auto_corr"] = None
df_good["fit_para"] = None
fits_para = []
mode_inter_spike_interval = []
for i in range(len(df_good)):
    autocorr_temp=[]
    hist_narrow=[]
    spike_times=df_good["spike_times"][:].iloc[i][:]
    _, autocorr_temp, bursting_index_temp = calculate_burst_index(spike_times)
    #hist_narrow, _, _ = calculate_burst_index(spike_times,bin_size=0.0005, max_lag=0.05)
    #temp_fits = fit_ACG(hist_narrow, plots=True)\hist_narrow
    hist_narrow = acg_narrow(spike_times)
    fits_para.append(hist_narrow)
    bursting_index.append(bursting_index_temp)
    auto_corr.append(autocorr_temp)
    mean_firing_rate.append(calculate_mean_firing_rate(spike_times)) 
    isis = np.diff(spike_times)
    if (df_good['peak_to_valley'].iloc[i] <= 0.000425)|(df_good['mean_firing_rate'].iloc[i] >= 10):
        df_good['buzaki_cell_type'].iloc[i] = "narrow_spike_interneurons"
    elif  (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['matlab_acg_tau_rise_1'].iloc[i]> 6):
        df_good['buzaki_cell_type'].iloc[i] = "wide_spike_interneurons"
    elif (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['mean_firing_rate'].iloc[i] < 10):
        df_good['buzaki_cell_type'].iloc[i] = "pyramidal"
    # Compute autocorre
    mode_inter_spike_interval.append(mode(isis, keepdims=True)[0][0])
np.save(r"Q:\sachuriga\CR_CA1_paper\tables/narrow_acgs.npy",fits_para)
df_good["auto_corr"] =  auto_corr
df_good["bursting_index"] = bursting_index
df_good["mean_firing_rate"] = mean_firing_rate
df_good["mode_inter_spike_interval"] = mode_inter_spike_interval
#df_good["fit_para"] =  fits_para
df_good.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

In [ ]:
#df_good = pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/all_units_table_with_waveforms.pkl")
df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
fits_para=[]
for i in range(len(df_good)):
    hist_narrow=[]
    spike_times=df_good["spike_times"][:].iloc[i][:]
    hist_narrow = acg_narrow(spike_times)
    fits_para.append(hist_narrow)

narrow_acgs = np.array(fits_para).T
para = fit_ACG(narrow_acgs, plots=False)
df_good['tau_rise_python']=None
df_good['tau_rise_python']=para['acg_tau_rise']
df_good['acg_first_positive_x']=None
df_good['acg_first_positive_x']=para['acg_first_positive_x']

df_good['buzaki_py_cell_type']=None
for i in range(len(df_good)):
    if (df_good['peak_to_valley'].iloc[i] <= 0.000425)|(df_good['mean_firing_rate'].iloc[i] >= 10):
        df_good['buzaki_py_cell_type'].iloc[i] = "narrow_spike_interneurons"
    #elif  (df_good['peak_to_valley'].iloc[i] > 0.000425) & (((df_good['tau_rise_python'].iloc[i]) + (df_good['acg_first_positive_x'].iloc[i])) >= 6):
    elif  (df_good['peak_to_valley'].iloc[i] > 0.000425) & ((df_good['tau_rise_python'].iloc[i])  >= 6):
        df_good['buzaki_py_cell_type'].iloc[i] = "wide_spike_interneurons"
    #elif (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['mean_firing_rate'].iloc[i] < 10) & (((df_good['tau_rise_python'].iloc[i]) + (df_good['acg_first_positive_x'].iloc[i])) < 6):
    elif (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['mean_firing_rate'].iloc[i] < 10) & ((df_good['tau_rise_python'].iloc[i])  < 6):
        df_good['buzaki_py_cell_type'].iloc[i] = "pyramidal"

#df_good.to_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/all_units_table_with_waveforms.pkl")
df_good.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

In [ ]:
narrow_acgs = np.array(fits_para).T
para = fit_ACG(narrow_acgs, plots=False)
df_good['tau_rise_python']=None
df_good['tau_rise_python']=para['acg_tau_rise']
df_good['acg_first_positive_x']=None
df_good['acg_first_positive_x']=para['acg_first_positive_x']

In [ ]:
narrow_acgs = np.array(fits_para).T
narrow_acgs.shape

In [ ]:
import numpy as np
import pandas as pd

df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
len_check = 0
for u_id in list(np.unique(df_loaded['session_id'])):
    animal = u_id.split("_")[0]
    day= u_id.split("_")[1]
    session = u_id.split("_")[2]
    temp = df_loaded[df_loaded['session_id']==u_id]
    len_check = len_check + len(temp)

    if os.path.exists(fr"S:\Sachuriga\file_with_table\ripple_ch\{animal}_{day}_{session}_units_table_withDLC.pkl"):
        os.remove(fr"S:\Sachuriga\file_with_table\ripple_ch\{animal}_{day}_{session}_units_table_withDLC.pkl")
    temp.to_pickle(fr"S:\Sachuriga\file_with_table\ripple_ch\{animal}_{day}_{session}_units_table_withDLC.pkl")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from matplotlib.animation import FuncAnimation
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_good ['buzaki_cell_type'] = None
for i in range(len(df_good )):
    if (df_good['peak_to_valley'].iloc[i] <= 0.000425)|(df_good['mean_firing_rate'].iloc[i] >= 10):
        df_good['buzaki_cell_type'].iloc[i] = "narrow_spike_interneurons"
    elif  (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['matlab_acg_tau_rise_1'].iloc[i]> 6):
        df_good['buzaki_cell_type'].iloc[i] = "wide_spike_interneurons"
    elif (df_good['peak_to_valley'].iloc[i] > 0.000425) & (df_good['mean_firing_rate'].iloc[i] < 10):
        df_good['buzaki_cell_type'].iloc[i] = "pyramidal"

# Select features
##the original
features = ['mean_firing_rate', 'bursting_index',"mode_inter_spike_interval",'peak_to_valley','assy','peak_trough_ratio']
#features = ['peak_trough_ratio', 'mean_firing_rate', 'bursting_index',"mean_inter_spike_interval",'peak_to_valley']
#features = ['Averate_rate', 'bursting_index',"mode_inter_spike_interval",'assy','peak_to_valley']
### for testing
#features = ['peak_trough_ratio', 'mean_firing_rate', 'bursting_index',"mean_inter_spike_interval",'peak_to_valley']
X = df_good[features].values

# Step 1: Z-score the data
scaler = StandardScaler()
X_zscored = scaler.fit_transform(X)

# # Step 2: Apply t-SNE for dimensionality reduction to 3D
# group = df_good['genotype']
# NDNF-flp +/- and Pde1c +/-
# NDNF-flp-/- and Pde1c -/-

tsne = TSNE(n_components=3, perplexity=30, n_iter=2000, random_state=42)
X_tsne = tsne.fit_transform(X_zscored)
# Assuming X_tsne and df_good are already defined from your previous steps
# If not, ensure X_tsne is a 3D t-SNE output (n_samples, 3)
df_loaded=df_good

In [ ]:
# Folder path
folder_path = r"S:\Sachuriga\file_with_table\ripple_ch\Functional_connections"

# Function to get pickle files
def get_pkl_files(folder_path):
    all_files = os.listdir(folder_path)
    pkl_files = [f for f in all_files if f.endswith("withDLC.pkl")]
    return pkl_files
pkl_files = get_pkl_files(folder_path)
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_loaded['connectivity']=None

for file in pkl_files:
    df = pd.read_pickle(fr'{folder_path}/{file}')
    session_id = df['session_id'].iloc[0]
    df_loaded.loc[df_loaded['session_id']==session_id,'connectivity'] = list(df['connectivity'])

df_good['connectivity'] = None
df_good['connectivity'] = df_loaded['connectivity'].values

In [ ]:
# Step 3: Apply DBSCAN clustering
#dbscan = DBSCAN(eps=1.4, min_samples=5)
#dbscan = DBSCAN(eps=1.1, min_samples=5)


dbscan = DBSCAN(eps=2.229, min_samples=20)
clusters = dbscan.fit_predict(X_tsne)

# Step 4: Identify the three largest clusters
unique_labels = np.unique(clusters)
cluster_sizes = np.bincount(clusters + 1)  # +1 to shift -1 (noise) to 0
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)  # Exclude noise

# Get sizes for non-noise clusters (exclude index 0, which is noise)
non_noise_sizes = cluster_sizes[1:]  # Skip noise (-1, now at index 0)
non_noise_labels = unique_labels[unique_labels != -1]  # Exclude noise label

# Sort clusters by size and pick the top 3
if len(non_noise_labels) >= 3:
    sorted_indices = np.argsort(non_noise_sizes)[::-1]  # Descending order
    largest_cluster_label = non_noise_labels[sorted_indices[0]]
    second_largest_cluster_label = non_noise_labels[sorted_indices[1]]
    third_largest_cluster_label = non_noise_labels[sorted_indices[2]]
else:
    raise ValueError("Fewer than 3 clusters found. Adjust DBSCAN parameters.")

##Filter data for the three largest clusters and noise
mask_largest = (clusters == largest_cluster_label) 
mask_second = clusters == second_largest_cluster_label
mask_third = clusters == third_largest_cluster_label
# mask_largest = df_loaded['buzaki_cell_type']=='pyramidal'
# mask_second = df_loaded['buzaki_cell_type']=="narrow_spike_interneurons"
# mask_third = df_loaded['buzaki_cell_type']=="wide_spike_interneurons"

mask_pre = list(df_good['connectivity']=="pre") & mask_largest
mask_post_n = list(df_good['connectivity']=="post") & (mask_second)
mask_post_w = list(df_good['connectivity']=="post") & (mask_third)


mask_noise = clusters == -1  # Noise points
X_largest_nonpre = (mask_largest == True) & (mask_pre==False)
X_second_nonpost = (mask_second == True) & (mask_post_n==False)
X_third_nonpost = (mask_third == True) & (mask_post_w==False)

X_largest = X_tsne[X_largest_nonpre] 
X_second = X_tsne[X_second_nonpost]
X_third = X_tsne[X_third_nonpost]

X_noise = X_tsne[mask_noise]
# Step 5: Assign labels to the original DataFrame
df_good['cell_type'] = 'noise'  # Default label for all points
df_good.loc[clusters == largest_cluster_label, 'cell_type'] = 'pyramidal'  # Largest cluster
df_good.loc[clusters == second_largest_cluster_label, 'cell_type'] = 'interneuron'  # Second largest
df_good.loc[clusters == third_largest_cluster_label, 'cell_type'] = 'wid_spike_interneuron'  # Third largest
color_hue = df_good['Information_content_rate'][df_good['cell_type']=='pyramidal']

# --- 3D Spinning GIF ---
fig_3d = plt.figure(figsize=(12, 12))
ax_3d = fig_3d.add_subplot(111, projection='3d')

# Plot largest cluster in blue (circle)
scatter1 = ax_3d.scatter(X_largest[:, 0], X_largest[:, 1], X_largest[:, 2], c='blue', s=30, label='Pyramidal', marker='o')
# Plot second largest cluster in red (circle)
scatter2 = ax_3d.scatter(X_second[:, 0], X_second[:, 1], X_second[:, 2], c='red', s=30, label='Interneuron', marker='o')
# Plot third largest cluster in green (circle)
scatter3 = ax_3d.scatter(X_third[:, 0], X_third[:, 1], X_third[:, 2], c='green', s=10, label='Wid Interneuron?', marker='o')

# Overlay mask_pre with triangle marker
mask_pre_indices = df_good[mask_pre].index
X_pre = X_tsne[mask_pre]

# Overlay mask_post with rectangle (square) marker
mask_post_indices = df_good[mask_post_n].index
X_post_n = X_tsne[mask_post_n]
scatter_post = ax_3d.scatter(X_post_n[:, 0], X_post_n[:, 1], X_post_n[:, 2], c='red', s=50, marker='s', alpha=1, edgecolors='black')

# Overlay mask_post with rectangle (square) marker
mask_post_indices = df_good[mask_post_w].index
X_post_w = X_tsne[mask_post_w]
scatter_post = ax_3d.scatter(X_post_w[:, 0], X_post_w[:, 1], X_post_w[:, 2], c='green', s=50, marker='s', alpha=1, edgecolors='black')
scatter_pre = ax_3d.scatter(X_pre[:, 0], X_pre[:, 1], X_pre[:, 2], c='blue', s=50, marker='^', alpha=1, edgecolors='black')

# Customize 3D plot
ax_3d.set_title('t-SNE 3D Projection: Three Largest DBSCAN Clusters with Noise').set_visible(False)
ax_3d.set_xlabel('t-SNE 1')
ax_3d.set_ylabel('t-SNE 2')
ax_3d.set_zlabel('t-SNE 3')
ax_3d.legend().set_visible(False)
plt.show()

# # Animation function to rotate the plot
# def update(frame):
#     ax_3d.view_init(elev=15, azim=frame)  # Rotate around z-axis
#     return scatter1, scatter2, scatter3,scatter_post,scatter_pre

# # Create animation
# ani = FuncAnimation(fig_3d, update, frames=np.arange(0, 360, 2), interval=50, blit=True)

# # Save the animation as a GIF
# ani.save(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/spinning_3d_clusters_with_noise.gif', writer='pillow', fps=10)
# plt.close(fig_3d)  # Close the figure to avoid displaying it statically

# --- 2D Static Plot ---
fig_2d = plt.figure(figsize=(10, 10))
ax_2d = fig_2d.add_subplot(111)

# Plot largest cluster in blue (only t-SNE 1 and t-SNE 2)
ax_2d.scatter(X_largest[:, 0], X_largest[:, 1], c='blue', s=20, label='Cluster Pyramidal (Largest)')
# Plot second largest cluster in red (only t-SNE 1 and t-SNE 2)
ax_2d.scatter(X_second[:, 0], X_second[:, 1], c='red', s=20, label='Cluster Interneuron (Second Largest)')
# Plot third largest cluster in green (only t-SNE 1 and t-SNE 2)
ax_2d.scatter(X_third[:, 0], X_third[:, 1], c='green', s=20, label='Cluster Third Largest')
# Plot noise in yellow (only t-SNE 1 and t-SNE 2)
#ax_2d.scatter(X_noise[:, 0], X_noise[:, 1], c='yellow', s=20, label='Noise')

scatter_pre = ax_2d.scatter(X_pre[:, 0], X_pre[:, 1], c='blue', s=50, marker='^', edgecolors='black')
scatter_post = ax_2d.scatter(X_post_n[:, 0], X_post_n[:, 1], c='red', s=50, marker='s', edgecolors='black')
scatter_post = ax_2d.scatter(X_post_w[:, 0], X_post_w[:, 1], c='green', s=50, marker='s', edgecolors='black')

# Customize 2D plot
ax_2d.set_title('t-SNE 2D Projection: Three Largest DBSCAN Clusters with Noise').set_visible(False)
ax_2d.set_xlabel('t-SNE 1')
ax_2d.set_ylabel('t-SNE 2')
ax_2d.legend().set_visible(False)

plt.axis('equal') # Ensures equal aspect ratio
# Adjust layout and save/show 2D plot
plt.tight_layout()
# plt.savefig(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/2d_clusters_with_noise.png')  # Save as PNG
plt.show()

# ## change labels to pkl files
# # Function to get pickle files
# def get_pkl_files(folder_path):
#     all_files = os.listdir(folder_path)
#     pkl_files = [f for f in all_files if f.endswith("withDLC.pkl")]
#     return pkl_files

# folder_path = r"S:\Sachuriga\file_with_table\ripple_ch"
# pkl_files = get_pkl_files(folder_path)

# # Initialize df_good (assuming it's a DataFrame)
# df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
# error_log=[]
# for file in pkl_files:
#     # Read the pickle file
#     try:
#         df = pd.read_pickle(rf"{folder_path}/{file}")
#     except Exception as e:
#         error_log.append(file)
#         continue
#     uids = df['session_id'].iloc[0]
    
#     temp = df_good[df_good['session_id']==uids]
#     df['cell_type']=None
#     df['cell_type'] = temp['cell_type'].tolist()
#     if os.path.exists(rf"{folder_path}/{file}"):
#         os.remove(rf"{folder_path}/{file}")
#     df.to_pickle(rf"{folder_path}/{file}")

# df_good.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

# Print cluster summary
print(f"Number of clusters found: {n_clusters}")
print(f"Number of noise points: {np.sum(clusters == -1)}")
print(f"Size of largest cluster (label {largest_cluster_label}): {np.sum(mask_largest)}")
print(f"Size of second largest cluster (label {second_largest_cluster_label}): {np.sum(mask_second)}")
print(f"Size of third largest cluster (label {third_largest_cluster_label}): {np.sum(mask_third)}")

In [ ]:
import pandas as pd
folder_path = r"S:\Sachuriga\file_with_table\ripple_ch"
# Initialize df_good (assuming it's a DataFrame)
df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
error_log=[]
for uid in df_good['session_id']:
    # Read the pickle file
    temp = df_good[df_good['session_id']==uid]
    file_name = uid.split('_phy_k_manual.nwb')[0]
    if os.path.exists(rf"{folder_path}/{file_name}_units_table_withDLC.pkl"):
        os.remove(rf"{folder_path}/{file_name}_units_table_withDLC.pkl")
    temp.to_pickle(rf"{folder_path}/{file_name}_units_table_withDLC.pkl")

In [ ]:
error_log=[]
folder_path=r'S:\Sachuriga\file_with_table\ripple_ch'
for file in pkl_files:
    # Read the pickle file
    try:
        df = pd.read_pickle(rf"{folder_path}/{file}")
    except Exception as e:
        error_log.append(file)
        continue
    uids = df['session_id'].iloc[0]
    
    temp = df_good[df_good['session_id']==uids]
    df['cell_type']=None
    df['cell_type'] = temp['cell_type'].tolist()
    if os.path.exists(rf"{folder_path}/{file}"):
        os.remove(rf"{folder_path}/{file}")
    df.to_pickle(rf"{folder_path}/{file}")

In [ ]:
all_units_df = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
assy=[]
for wave in all_units_df['waveform']:
    a = np.max(wave[:29])
    b = np.max(wave[29:])
    temp = (b-a)/(a+b)
    assy.append(temp)
all_units_df['assy']=None
all_units_df['assy']=assy
all_units_df['cell_type']=None
all_units_df['buzaki_cell_type']=None

all_units_df.loc[:,"assy"] = assy
all_units_df.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import numpy as np

# Assuming df_good, X_tsne, mask, and other variables are defined
mask = (df_good['cell_type'] == 'pyramidal') & (df_good['genotype'] == 'NDNF-flp-/- and Pde1c -/-') 
#mask = (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype'] == 'NDNF-flp-/- and Pde1c -/-')
#mask = (df_good['cell_type'] == 'pyramidal') & (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype'] == 'NDNF-flp-/- and Pde1c -/-')
#color_hue = df_good['Averate_rate'][mask]  # Use 'Averate_rate' as hue

color_hue = df_good['Averate_rate'][mask]  # Use 'Averate_rate' as hue
X_largest = X_tsne[mask, :]  # 3D t-SNE data

# Create figure and 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax.scatter(X_largest[:, 0], X_largest[:, 1], X_largest[:, 2], 
                    c=color_hue, cmap='hot', vmin=0, vmax=8)

# Set labels
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_zlabel('t-SNE 3')
ax.legend().set_visible(False)

# Add color bar with range 0 to 12
cbar = plt.colorbar(scatter, label='Averate_rate')


# Animation function to rotate the plot
def update(frame):
    ax.view_init(elev=15, azim=frame)  # Rotate around z-axis
    return scatter,

# Create animation
ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=50, blit=True)

# Save the animation as a GIF
ani.save(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/spinning_3d_clusters_CRs-_spi.gif', writer='pillow', fps=10)

# Show plot
plt.show()

# Create figure and 3D axes
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(111)

# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax1.scatter(X_largest[:, 0], X_largest[:, 1], 
                    c=color_hue, cmap='hot', vmin=0, vmax=8, s=100)
ax1.set_xlabel('t-SNE 1')
ax1.set_ylabel('t-SNE 2')
ax1.set_xlim([-18, 15])
ax1.set_ylim([-20, 15])


high_mask = (df_good['cell_type'] == 'pyramidal') &  (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')  &  (df_good['Averate_rate']>2) 
low_mask = (df_good['cell_type'] == 'pyramidal') &  (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')  &  (df_good['Averate_rate']<=2) 

# Create a DataFrame similar to iris, combining superficial and deep data
data_superficial = pd.DataFrame({
    'tSNE_1': X_tsne[high_mask, 0],
    'tSNE_2': X_tsne[high_mask, 1],
    'sub_population': 'high rate'
})
data_deep = pd.DataFrame({
    'tSNE_1': X_tsne[low_mask, 0],
    'tSNE_2': X_tsne[low_mask, 1],
    'sub_population': 'low rate'
})
df_plot = pd.concat([data_superficial, data_deep], ignore_index=True)

# Set up the figure
#f, ax = plt.subplots(figsize=(8, 8))
ax1.set_aspect("equal")

# Draw a contour plot to represent each bivariate density
sns.kdeplot(
    data=df_plot,
    x="tSNE_1",
    y="tSNE_2",
    hue="sub_population",
    palette=['red', 'black'],
    thresh=0.1,
    ax=ax1
)
plt.axis('equal')
# Show plot
plt.show()


In [ ]:
#mask=(df_good['cell_type'] == 'pyramidal') & (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')
mask=(df_good['cell_type'] == 'pyramidal') &  (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-') 
#mask=(df_good['cell_type'] == 'pyramidal') &  (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-') 
#color_hue = df_good['Averate_rate'][mask]  # Use 'Averate_rate' as hue


color_hue = df_good['Averate_rate'][mask]  # Use 'Averate_rate' as hue
X_largest = X_tsne[mask, :]  # 3D t-SNE data

# Create figure and 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax.scatter(X_largest[:, 0], X_largest[:, 1], X_largest[:, 2], 
                    c=color_hue, cmap='hot', vmin=0, vmax=8)

# Set labels
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_zlabel('t-SNE 3')
ax.legend().set_visible(False)

# Add color bar with range 0 to 12
cbar = plt.colorbar(scatter, label='Averate_rate')


# Animation function to rotate the plot
def update(frame):
    ax.view_init(elev=15, azim=frame)  # Rotate around z-axis
    return scatter,

# Create animation
ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=50, blit=True)

# Save the animation as a GIF
ani.save(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/spinning_3d_clusters_CRs+_spi.gif', writer='pillow', fps=10)
# Show plot
plt.show()


# Create figure and 3D axes
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(111)

# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax1.scatter(X_largest[:, 0], X_largest[:, 1], 
                    c=color_hue, cmap='hot', vmin=0, vmax=8, s=100)
ax1.set_xlabel('t-SNE 1')
ax1.set_ylabel('t-SNE 2')
ax1.set_xlim([-18, 15])
ax1.set_ylim([-20, 15])

high_mask = (df_good['cell_type'] == 'pyramidal') &  (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')  &  (df_good['Averate_rate']>2) 
low_mask = (df_good['cell_type'] == 'pyramidal') &  (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')  &  (df_good['Averate_rate']<=2) 

# Create a DataFrame similar to iris, combining superficial and deep data
data_superficial = pd.DataFrame({
    'tSNE_1': X_tsne[high_mask, 0],
    'tSNE_2': X_tsne[high_mask, 1],
    'sub_population': 'high rate'
})
data_deep = pd.DataFrame({
    'tSNE_1': X_tsne[low_mask, 0],
    'tSNE_2': X_tsne[low_mask, 1],
    'sub_population': 'low rate'
})
df_plot = pd.concat([data_superficial, data_deep], ignore_index=True)

# Set up the figure
#f, ax = plt.subplots(figsize=(8, 8))
ax1.set_aspect("equal")

# Draw a contour plot to represent each bivariate density
sns.kdeplot(
    data=df_plot,
    x="tSNE_1",
    y="tSNE_2",
    hue="sub_population",
    palette=['red', 'black'],
    thresh=0.1,
    ax=ax1
)

# Show plot
plt.show()

In [ ]:
#mask=(df_good['cell_type'] == 'pyramidal') & (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')
mask_sup=(df_good['cell_type'] == 'pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-') & (df_good['sub_population'] == 'superficial') 
mask_deep=(df_good['cell_type'] == 'pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-') & (df_good['sub_population'] == 'deep')


X_superficial = X_tsne[mask_sup, :]  # 3D t-SNE data
X_deep = X_tsne[mask_deep, :] 

# Create figure and 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax.scatter(X_superficial[:, 0], X_superficial[:, 1], X_superficial[:, 2], c='blue', s=20)
scatter = ax.scatter(X_deep[:, 0], X_deep[:, 1], X_deep[:, 2], c='black', s=20)

# Set labels
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_zlabel('t-SNE 3')
ax.legend().set_visible(False)


# Animation function to rotate the plot
def update(frame):
    ax.view_init(elev=15, azim=frame)  # Rotate around z-axis
    return scatter,

# Create animation
ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=50, blit=True)
# Save the animation as a GIF
ani.save(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/spinning_3d_clusters_CRs+_subpopulation.gif', writer='pillow', fps=10)
# Show plot
plt.show()
# Create figure and 3D axes
color1 = sns.color_palette("hls", 8)[1]
color2 = sns.color_palette("hls", 8)[-2]

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(111)
#mask=(df_good['cell_type'] == 'pyramidal') & (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')
mask_sup=(df_good['cell_type'] == 'pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-') & (df_good['sub_population'] == 'superficial') 
mask_deep=(df_good['cell_type'] == 'pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-') & (df_good['sub_population'] == 'deep')

# Scatter plot with 3D coordinates and color based on color_hue
# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax1.scatter(X_superficial[:, 0],X_superficial[:, 1], c=color1, s=20)
scatter = ax1.scatter(X_deep[:, 0],X_deep[:, 1], c=color2, s=20)

ax1.set_xlabel('t-SNE 1')
ax1.set_ylabel('t-SNE 2')
ax1.set_xlim([-18, 15])
ax1.set_ylim([-20, 15])


# Create a DataFrame similar to iris, combining superficial and deep data
data_superficial = pd.DataFrame({
    'tSNE_1': X_superficial[:, 0],
    'tSNE_2': X_superficial[:, 1],
    'sub_population': 'superficial'
})
data_deep = pd.DataFrame({
    'tSNE_1': X_deep[:, 0],
    'tSNE_2': X_deep[:, 1],
    'sub_population': 'deep'
})
df_plot = pd.concat([data_superficial, data_deep], ignore_index=True)

sns.set_style("white")
# Set up the figure
#f, ax = plt.subplots(figsize=(8, 8))
ax1.set_aspect("equal")

# Draw a contour plot to represent each bivariate density
sns.kdeplot(
    data=df_plot,
    x="tSNE_1",
    y="tSNE_2",
    hue="sub_population",
    palette=[color1, color2],
    thresh=0.1,
    ax=ax1
)
# Set transparent background
fig.patch.set_facecolor('none')  # Transparent figure background
ax1.set_facecolor('none')      # Transparent axes background
plt.axis('equal')
plt.grid(False)
# Show plot
plt.show()


In [ ]:
#mask=(df_good['cell_type'] == 'pyramidal') & (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')
mask_sup=(df_good['cell_type'] == 'pyramidal') & (df_good['genotype'] == 'NDNF-flp-/- and Pde1c -/-') & (df_good['sub_population'] == 'superficial') 
mask_deep=(df_good['cell_type'] == 'pyramidal') & (df_good['genotype'] == 'NDNF-flp-/- and Pde1c -/-') & (df_good['sub_population'] == 'deep') 
 

X_superficial = X_tsne[mask_sup, :]  # 3D t-SNE data
X_deep = X_tsne[mask_deep, :] 

# Create figure and 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax.scatter(X_superficial[:, 0], X_superficial[:, 1], X_superficial[:, 2], c='blue')
scatter = ax.scatter(X_deep[:, 0], X_deep[:, 1], X_deep[:, 2], c='black')

# Set labels
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_zlabel('t-SNE 3')
ax.legend().set_visible(False)


# Animation function to rotate the plot
def update(frame):
    ax.view_init(elev=15, azim=frame)  # Rotate around z-axis
    return scatter,

# Create animation
ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=50, blit=True)

# Save the animation as a GIF
ani.save(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/spinning_3d_clusters_CRs+_subpopulation.gif', writer='pillow', fps=10)
# Show plot
plt.show()


# Create figure and 3D axes
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(111)

# Scatter plot with 3D coordinates and color based on color_hue
# Scatter plot with 3D coordinates and color based on color_hue
scatter = ax1.scatter(X_superficial[:, 0],X_superficial[:, 1], c=color1, s=20)
scatter = ax1.scatter(X_deep[:, 0],X_deep[:, 1], c=color2, s=20)

ax1.set_xlabel('t-SNE 1')
ax1.set_ylabel('t-SNE 2')
ax1.set_xlim([-18, 15])
ax1.set_ylim([-20, 15])



# Create a DataFrame similar to iris, combining superficial and deep data
data_superficial = pd.DataFrame({
    'tSNE_1': X_superficial[:, 0],
    'tSNE_2': X_superficial[:, 1],
    'sub_population': 'superficial'
})
data_deep = pd.DataFrame({
    'tSNE_1': X_deep[:, 0],
    'tSNE_2': X_deep[:, 1],
    'sub_population': 'deep'
})
df_plot = pd.concat([data_superficial, data_deep], ignore_index=True)

# Set up the figure
#f, ax = plt.subplots(figsize=(8, 8))
ax1.set_aspect("equal")

# Draw a contour plot to represent each bivariate density
sns.kdeplot(
    data=df_plot,
    x="tSNE_1",
    y="tSNE_2",
    hue="sub_population",
    thresh=0.1,
    palette=[color1, color2],
    ax=ax1
)
# Set transparent background
fig.patch.set_facecolor('none')  # Transparent figure background
ax1.set_facecolor('none')      # Transparent axes background
plt.axis('equal')
plt.grid(False)
# Show plot
plt.show()

In [ ]:
#mask=(df_good['cell_type'] == 'pyramidal') & (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')
mask= (df_good['buzaki_cell_type']=='pyramidal') & (df_good['genotype']=='NDNF-flp +/- and Pde1c +/-')
color_hue = df_good['Information_content_rate'][mask]

import matplotlib.pyplot as plt
import seaborn as sns  # For better color handling with hue

# Create figure and axis
fig, ax = plt.subplots()

# Assuming df_good, X_largest, and X_second_largest are defined
# color_hue for pyramidal cells
color_hue = df_good['Information_content_rate'][mask]

X_largest = X_tsne[mask,:]
# Plot largest cluster in blue with hue based on Information_content_rate
sns.scatterplot(x=X_largest[:, 0], y=X_largest[:, 1], hue=color_hue, palette='cool',
                ax=ax)

# # Plot second largest cluster in red (assuming X_second_largest is defined)
# sns.scatterplot(x=X_second_largest[:, 0], y=X_second_largest[:, 1], color='red', 
#                 label='Cluster Pyramidal (Second Largest)', ax=ax)

# Add labels and legend
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.legend()

# Show plot
plt.show()

In [ ]:
# Assuming X_tsne and df_good are already defined from your previous steps
# Step 3 and beyond remain unchanged until plotting

# --- 3D Spinning GIF ---
fig_3d = plt.figure(figsize=(12, 12))
ax_3d = fig_3d.add_subplot(111, projection='3d')

# Define conditions for plotting based on genotype and cell_type
exp = "NDNF-flp +/- and Pde1c +/-"
control = "NDNF-flp-/- and Pde1c -/-"

# Masks for genotype and cell type combinations
mask_control_pyramidal = (df_good['genotype'] == control) & (df_good['cell_type'] == 'pyramidal')
mask_exp_pyramidal = (df_good['genotype'] == exp) & (df_good['cell_type'] == 'pyramidal')
mask_control_interneuron = (df_good['genotype'] == control) & (df_good['cell_type'] == 'interneuron')
mask_exp_interneuron = (df_good['genotype'] == exp) & (df_good['cell_type'] == 'interneuron')

mask_control_interneuron_w = (df_good['genotype'] == control) & (df_good['cell_type'] == 'wid_spike_interneuron')
mask_exp_interneuron_w = (df_good['genotype'] == exp) & (df_good['cell_type'] == 'wid_spike_interneuron')
mask_noise = clusters == -1  # Noise points

# # Masks for genotype and cell type combinations
# mask_control_pyramidal = (df_good['genotype'] == control) & (df_loaded['buzaki_cell_type'] == 'pyramidal')
# mask_exp_pyramidal = (df_good['genotype'] == exp) & (df_loaded['buzaki_cell_type'] == 'pyramidal')
# mask_control_interneuron = (df_good['genotype'] == control) & (df_loaded['buzaki_cell_type'] == 'narrow_spike_interneurons')
# mask_exp_interneuron = (df_good['genotype'] == exp) & (df_loaded['buzaki_cell_type'] == 'narrow_spike_interneurons')


# Extract t-SNE coordinates for each group
X_control_pyramidal = X_tsne[mask_control_pyramidal]
X_exp_pyramidal = X_tsne[mask_exp_pyramidal]
X_control_interneuron = X_tsne[mask_control_interneuron]
X_exp_interneuron = X_tsne[mask_exp_interneuron]
X_control_interneuron_w = X_tsne[mask_control_interneuron_w]
X_exp_interneuron_w = X_tsne[mask_exp_interneuron_w]


X_noise = X_tsne[mask_noise]

# Plot with specified colors
scatter1 = ax_3d.scatter(X_control_pyramidal[:, 0], X_control_pyramidal[:, 1], X_control_pyramidal[:, 2], 
                         c='blue', s=30, label='Control Pyramidal')
scatter2 = ax_3d.scatter(X_exp_pyramidal[:, 0], X_exp_pyramidal[:, 1], X_exp_pyramidal[:, 2], 
                         c='cyan', s=30, label='Exp Pyramidal')
scatter3 = ax_3d.scatter(X_control_interneuron[:, 0], X_control_interneuron[:, 1], X_control_interneuron[:, 2], 
                         c='red', s=30, label='Control Interneuron')
scatter4 = ax_3d.scatter(X_exp_interneuron[:, 0], X_exp_interneuron[:, 1], X_exp_interneuron[:, 2], 
                         c='magenta', s=30, label='Exp Interneuron')
scatter5 = ax_3d.scatter(X_control_interneuron_w[:, 0], X_control_interneuron_w[:, 1], X_control_interneuron_w[:, 2], 
                         c='green', s=30, label='Control Interneuron')
scatter6 = ax_3d.scatter(X_exp_interneuron_w[:, 0], X_exp_interneuron_w[:, 1], X_exp_interneuron_w[:, 2], 
                         c='lime', s=30, label='Exp Interneuron')


# scatter5 = ax_3d.scatter(X_noise[:, 0], X_noise[:, 1], X_noise[:, 2], 
#                          c='yellow', s=10, label='Noise')

# Customize 3D plot
ax_3d.set_title('t-SNE 3D Projection: Genotype and Cell Type Classification').set_visible(False)
ax_3d.set_xlabel('t-SNE 1')
ax_3d.set_ylabel('t-SNE 2')
ax_3d.set_zlabel('t-SNE 3')
ax_3d.legend().set_visible(False)

# Animation function to rotate the plot
def update(frame):
    ax_3d.view_init(elev=15, azim=frame)  # Rotate around z-axis
    return scatter1, scatter2, scatter3, scatter4, scatter5, scatter6

# Create animation
ani = FuncAnimation(fig_3d, update, frames=np.arange(0, 360, 2), interval=50, blit=True)

# Save the animation as a GIF
ani.save(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/spinning_3d_genotype_celltype.gif', writer='pillow', fps=10)
plt.close(fig_3d)  # Close the figure to avoid displaying it statically

# --- 2D Static Plot ---
fig_2d = plt.figure(figsize=(10, 10))
ax_2d = fig_2d.add_subplot(111)

# Plot with specified colors (2D projection)
ax_2d.scatter(X_control_pyramidal[:, 0], X_control_pyramidal[:, 1], c='blue', s=20, label='Control Pyramidal')
ax_2d.scatter(X_exp_pyramidal[:, 0], X_exp_pyramidal[:, 1], c='cyan', s=20, label='Exp Pyramidal')
ax_2d.scatter(X_control_interneuron[:, 0], X_control_interneuron[:, 1], c='red', s=20, label='Control Interneuron')
ax_2d.scatter(X_exp_interneuron[:, 0], X_exp_interneuron[:, 1], c='magenta', s=20, label='Exp Interneuron')
ax_2d.scatter(X_noise[:, 0], X_noise[:, 1], c='yellow', s=20, label='Noise')

# Customize 2D plot
ax_2d.set_title('t-SNE 2D Projection: Genotype and Cell Type Classification')
ax_2d.set_xlabel('t-SNE 1')
ax_2d.set_ylabel('t-SNE 2')
ax_2d.legend()

plt.axis('equal')  # Ensures equal aspect ratio
# Adjust layout and save/show 2D plot
plt.tight_layout()
plt.savefig(r'Q:\sachuriga/CR_CA1_paper/Results/Cell_type/2d_genotype_celltype.png')  # Save as PNG
plt.show()

# Save the DataFrame
# df_good.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/good_units_with_tsnLabels.pkl')

# Print summary (unchanged)
print(f"Number of clusters found: {n_clusters}")
print(f"Number of noise points: {np.sum(clusters == -1)}")
print(f"Size of largest cluster (label {largest_cluster_label}): {np.sum(mask_largest)}")
print(f"Size of second largest cluster (label {second_largest_cluster_label}): {np.sum(mask_second)}")
print(f"Size of third largest cluster (label {third_largest_cluster_label}): {np.sum(mask_third)}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Load Good units
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_loaded=df_good
df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df_py = df_good[df_good['buzaki_py_cell_type']=='pyramidal']
df = df_py
base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
session = ["A", "B", "C", "Total"]

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# ... (Your existing code for loading data, filtering, and setting up the figure remains unchanged)
test_hy = ['greater','less','greater','less','less','two-sided']
for session in session:
    # Filter for sessions
    if session == "Total":
        df_a = df
    else:
        df_a = df[df['session'] == session]

    # Separate into control and experimental groups
    control_df = df_a[df_a['animal_id'].isin(control_ids)]
    exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

    # Set Seaborn theme
    sns.set_theme(style="ticks")

    # Statistical comparisons for scalar metrics
    metrics = ['Information_content_rate', 'Sparsity', 'Selectivity', 'Averate_rate', 'Field_size', 'matlab_stability_smooth1']

    # Create figure with 2 rows and 6 columns
    fig, axes = plt.subplots(2, 6, figsize=(18, 10), width_ratios=[1, 0.5, 1, 0.5, 1, 0.5])
    axes = axes.flatten()  # Flatten for easier iteration

    # Define custom colors
    control_color = sns.color_palette(palette='flag')[-1]  # Dark blue for Control
    exp_color = "cyan"  # Light blue for Experimental

    for idx, metric in enumerate(metrics):
        control_values = control_df[metric].dropna()
        exp_values = exp_df[metric].dropna()
        
        if len(control_values) > 0 and len(exp_values) > 0:
            control_mean = control_values.mean()
            exp_mean = exp_values.mean()
            control_sem = control_values.sem()
            exp_sem = exp_values.sem()
            
            print(f"\nComparison for {metric}:")
            print(f"Control mean: {control_mean:.2f} ± {control_sem:.2f}")
            print(f"Experimental mean: {exp_mean:.2f} ± {exp_sem:.2f}")
            
            # Mann-Whitney U test
            control_array = np.asarray(control_values.values, dtype=float)
            exp_array = np.asarray(exp_values.values, dtype=float)

            # Remove NaN values
            control_clean = control_array[~np.isnan(control_array)]
            exp_clean = exp_array[~np.isnan(exp_array)]

            # Perform Mann-Whitney U test
            #u_stat, p_val = stats.mannwhitneyu(control_clean, exp_clean, alternative='two-sided')

            # Step 1: Test normality
            shapiro_control = stats.shapiro(control_clean)
            shapiro_exp = stats.shapiro(exp_clean)
            print("Shapiro-Wilk Test - Control: p-value =", shapiro_control.pvalue)
            print("Shapiro-Wilk Test - Experimental: p-value =", shapiro_exp.pvalue)

            # Step 2: Calculate means and standard deviations
            print("Control Mean:", np.mean(control_clean), "SD:", np.std(control_clean, ddof=1))
            print("Experimental Mean:", np.mean(exp_clean), "SD:", np.std(exp_clean, ddof=1))

            # Step 3: Check variance equality
            levene_test = stats.levene(control_clean, exp_clean)
            print("Levene's Test for equal variances: p-value =", levene_test.pvalue)

            # Step 4: Choose and perform test
            alpha = 0.05
            if shapiro_control.pvalue > alpha and shapiro_exp.pvalue > alpha:
                print("Both groups are normally distributed.")
                if levene_test.pvalue > alpha:
                    print("Variances are equal. Performing two-sample t-test.")
                    t_stat, t_pval = stats.ttest_ind(control_clean, exp_clean, equal_var=True)
                    print("Two-sample t-test: t =", t_stat, "p-value =", t_pval)
                    if t_pval < alpha:
                        print("Reject H0: Means differ significantly.")
                    else:
                        print("Fail to reject H0: No significant difference in means.")
                else:
                    print("Variances are unequal. Consider Welch's t-test or Mann-Whitney U.")
                    u_stat, p_val = stats.ttest_ind(control_clean, exp_clean, equal_var=False)
                    print("Welch's t-test: t =", t_stat, "p-value =", t_pval)
                    if t_pval < alpha:
                        print("Reject H0: Means differ significantly.")
                    else:
                        print("Fail to reject H0: No significant difference in means.")
            else:
                print("At least one group is non-normal. Performing Mann-Whitney U test.")
                test_name = test_hy[idx]
                u_stat, p_val = stats.mannwhitneyu(control_clean, exp_clean, alternative=fr'{test_name}')
                print("Mann-Whitney U Test: U =", u_stat, "p-value =", p_val)
                if p_val < alpha:
                    print("Reject H0: Distributions differ significantly.")
                else:
                    print("Fail to reject H0: No significant difference in distributions.")
            
            # Prepare data for Seaborn plotting
            plot_df = pd.DataFrame({
                'value': pd.concat([control_values, exp_values]),
                'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
            })
            
            # Check if deviation is "too large" (using coefficient of variation > 1 as threshold)
            all_values = plot_df['value']
            cv = all_values.std() / all_values.mean()  # Coefficient of variation
            use_log_scale = cv > 1 and all_values.min() > 0  # Ensure positive values for log scale
            
            # Filter out outliers (e.g., beyond 3 standard deviations)
            mean_val = all_values.mean()
            std_val = all_values.std()
            plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                       (plot_df['value'] <= mean_val + 3 * std_val)]
            
            # Boxplot (left subplot for each metric)
            box_ax = axes[idx * 2]  # Left column for boxplot
            sns.boxplot(
                data=plot_df_filtered,
                x='group',
                y='value',
                ax=box_ax,
                palette={"Control": control_color, "Experimental": exp_color},
                width=0.5
            )
            
            # Add individual points with matching colors
            sns.stripplot(
                data=plot_df_filtered,
                x='group',
                y='value',
                ax=box_ax,
                size=4,
                hue='group',
                palette={"Control": "black", "Experimental": "black"},
                alpha=0.4,
                jitter=0.1,
                legend=False
            )
            
            # Set title and labels for boxplot
            box_ax.set_title(f'{metric} Comparison')
            box_ax.set_ylabel(metric)
            box_ax.set_xlabel('Group')
            box_ax.yaxis.grid(False)
            box_ax.set(xlabel="")
            
            # Add p-value at the top of the boxplot
            box_ax.text(0.5, 0.95, f'p = {p_val:.4f}', 
                        horizontalalignment='center', 
                        verticalalignment='top', 
                        transform=box_ax.transAxes, 
                        fontsize=10)
            
            # # Add N values above each boxplot bar
            # n_control = len(control_clean)
            # n_exp = len(exp_clean)
            # max_y = plot_df_filtered['value'].max()  # Get max y-value for positioning
            # y_offset = max_y * - 0.1  # Slightly above the max value
            # box_ax.text(0, y_offset, f'N={n_control}', 
            #             horizontalalignment='center', 
            #             fontsize=8, color=control_color)
            # box_ax.text(1, y_offset, f'N={n_exp}', 
            #             horizontalalignment='center', 
            #             fontsize=8, color=exp_color)
            
            # Remove top and right spines
            box_ax.spines['top'].set_visible(False)
            box_ax.spines['right'].set_visible(False)
            box_ax.spines['bottom'].set_visible(True)
            box_ax.spines['left'].set_visible(True)

            # KDE plot (right subplot for each metric) as probability density
            kde_ax = axes[idx * 2 + 1]  # Right column for KDE
            sns.kdeplot(
                data=plot_df_filtered,
                y='value',
                hue='group',
                ax=kde_ax,
                palette={"Control": control_color, "Experimental": exp_color},
                fill=True,
                alpha=0.3,
                common_norm=False,  # Each group's KDE integrates to 1
                legend=False
            )
            
            # Match y-axis limits with boxplot
            kde_ax.set_ylim(box_ax.get_ylim())
            kde_ax.set_xticks([])  # Remove x-axis ticks
            kde_ax.set_yticks([])  # Remove y-axis ticks
            kde_ax.set_xlabel('')
            kde_ax.set_ylabel('')
            
            # Remove spines for KDE plot
            kde_ax.spines['top'].set_visible(False)
            kde_ax.spines['right'].set_visible(False)
            kde_ax.spines['bottom'].set_visible(False)
            kde_ax.spines['left'].set_visible(False)

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# ... (Your existing code for loading data, filtering, and setting up variables remains unchanged)
# Assuming df, control_ids, exp_ids, and session are defined as in your original code

df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_py = df_good[df_good['buzaki_py_cell_type']=='pyramidal']
df = df_py
base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A"]


for session in sessions:
    # Filter for sessions
    if session == "Total":
        df_a = df
    else:
        df_a = df[df['session'] == session]

    # Separate into control and experimental groups
    control_df = df_a[df_a['animal_id'].isin(control_ids)]
    exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

    # Set Seaborn theme
    sns.set_theme(style="ticks")

    # Statistical comparisons for scalar metrics
    metrics = ['Information_content_rate', 'Sparsity', 'Selectivity', 'Averate_rate', 'Field_size', 'matlab_stability_smooth1']
    metrics_labels = ['Information content rate (spikes/bit)', 'Sparsity', 'Selectivity', 'Averate firing rate (Hz)', 'Field size (percentage)', 'Within session stability']
    # Create figure with 1 row and 6 columns (one per metric)
    fig, axes = plt.subplots(1, 6, figsize=(18, 5))
    axes = axes.flatten()  # Flatten for easier iteration

    # Define custom colors
    # control_color = sns.color_palette(palette='flag')[-1]  # Dark blue for Control
    # exp_color = "cyan"  # Light blue for Experimental
    control_color = '#2b4d5e'  # Dark blue for Control
    exp_color = '#BC554E'  # Light blue for Experimental

    i4label=0
    for idx, metric in enumerate(metrics):
        control_values = control_df[metric].dropna()
        exp_values = exp_df[metric].dropna()
        
        if len(control_values) > 0 and len(exp_values) > 0:
            control_mean = control_values.mean()
            control_sem = control_values.sem()
            exp_mean = exp_values.mean()
            exp_sem = exp_values.sem()
            
            print(f"\nComparison for {metric}:")
            print(f"Control mean: {control_mean:.2f} ± {control_sem:.2f}")
            print(f"Experimental mean: {exp_mean:.2f} ± {exp_sem:.2f}")
            
            # Mann-Whitney U test
            control_array = np.asarray(control_values.values, dtype=float)
            exp_array = np.asarray(exp_values.values, dtype=float)

            # Remove NaN values
            control_clean = control_array[~np.isnan(control_array)]
            exp_clean = exp_array[~np.isnan(exp_array)]

            # Perform Mann-Whitney U test
            u_stat, p_val = stats.mannwhitneyu(control_clean, exp_clean, alternative='two-sided')

            print(f"Mann-Whitney U statistic: {u_stat:.2f}, p-value: {p_val:.4f}")
            
            # Prepare data for Seaborn plotting
            plot_df = pd.DataFrame({
                'value': pd.concat([control_values, exp_values]),
                'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
            })
            
            # Check if deviation is "too large" (using coefficient of variation > 1 as threshold)
            all_values = plot_df['value']
            cv = all_values.std() / all_values.mean()  # Coefficient of variation
            use_log_scale = cv > 1 and all_values.min() > 0  # Ensure positive values for log scale
            
            # Filter out outliers (e.g., beyond 3 standard deviations)
            mean_val = all_values.mean()
            std_val = all_values.std()
            plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                       (plot_df['value'] <= mean_val + 3 * std_val)]
            
            # Bar plot
            ax = axes[idx]
            sns.barplot(
                data=plot_df_filtered,
                x='group',
                y='value',
                ax=ax,
                palette={"Control": control_color, "Experimental": exp_color},
                errorbar='se',  # Show SEM as error bars
                capsize=0.1,
                alpha=1,
            )
            
            # Add individual points with matching colors
            sns.stripplot(
                data=plot_df_filtered,
                x='group',
                y='value',
                ax=ax,
                size=2,
                hue='group',
                palette={"Control": "#ECE0CA", "Experimental": "#ECE0CA"},
                alpha=1,
                jitter=0.2,
                legend=False
            )
            
            # Set title and labels for bar plot
            #ax.set_title(f'{metric} Comparison')
            ax.set_ylabel(metrics_labels[idx])
            ax.set_xlabel('Group')
            ax.yaxis.grid(False)
            ax.set(xlabel="")
            ax.set_ylim(0)
            ax.set_xticklabels(['CRs +', 'CRs -'])
            # Add p-value at the top of the bar plot
            ax.text(0.5, 0.95, f'p = {p_val:.4f}', 
                    horizontalalignment='center', 
                    verticalalignment='top', 
                    transform=ax.transAxes, 
                    fontsize=10)
            
            # Remove top and right spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(True)
            ax.spines['left'].set_visible(True)
        i4label =+ 1

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

In [ ]:
import pynapple as nap
import numpy as np
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap_ax,plot_path
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,Speed_filtered_spikes

base_nwb_folder = fr"S:\Sachuriga\nwb\test4neo"
df=control_df.reset_index(drop=True)

sublayer = "superficial"
if sublayer:
    df = control_df[control_df['sub_population']==sublayer].reset_index(drop=True)
map_color = "jet"

for temp_name in np.unique(df['session_id']):
    print(temp_name)
    npdata = nap.load_file(fr"{base_nwb_folder }/{temp_name}")
    pos_cord = load_speed_fromNWB(npdata['XY_mid_brain'])

    raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(pos_cord[:,0], # times
                            pos_cord[:,1], # x
                            pos_cord[:,2], # y
                            filter_speed=True, 
                            min_speed = 0.05)
    
    

    temp_table = df[df['session_id']==temp_name]
    for i in range(len(temp_table)):

        spike_times = Speed_filtered_spikes(temp_table['spike_times'].iloc[i],
                            pos_cord[:,0],mask)
        
        maps = mapp.SpatialMap(box_size=[1.0, 1.0], bin_size=0.01, smoothing=0.1)
        rate_map = maps.rate_map(combined_array[:,1], combined_array[:,2], combined_array[:,0], spike_times)
        fig = plt.figure()
        ax = fig.add_subplot(111, xlim=[0, 1], ylim=[0, 1], aspect=1)
        ax.imshow(rate_map, interpolation='none', origin='upper',
                extent=(0, 1, 0, 1), vmin=0.01, cmap=fr'{map_color}')
        ax.grid(False)
                    # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        id=temp_table.index[i]
        ax.set_axis_off()
        if sublayer:
            fig.savefig(fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\control/{sublayer}/{map_color}/{id}.png", bbox_inches='tight', pad_inches=0, transparent=True)
        else:
            fig.savefig(fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\control/{map_color}/{id}.png", bbox_inches='tight', pad_inches=0, transparent=True)

import pandas as pd
import numpy as np

# 假设你的数据是一个Series
df = df.reset_index(drop=True)['Information_content_rate']
import pandas as pd
import numpy as np

def group_indices_by_step(df_col, step=0.075):
    used_indices = set()
    result = []
    values = df_col.copy()
    max_val = values.max()

    while len(used_indices) < len(values):
        remaining = values[~values.index.isin(used_indices)]
        if remaining.empty:
            break

        current_group = []
        current_val = remaining.min()
        current_idx = remaining.idxmin()
        current_group.append(current_idx)
        used_indices.add(current_idx)

        while True:
            target_val = current_val + step
            remaining = values[~values.index.isin(used_indices)]
            if remaining.empty:
                break

            # 找到大于等于 target_val 的值中最接近 target_val 的那个
            diffs = remaining - target_val
            diffs = diffs[diffs >= 0]
            if diffs.empty:
                break

            next_idx = diffs.idxmin()
            current_val = values[next_idx]
            current_group.append(next_idx)
            used_indices.add(next_idx)

        result.append(current_group)

    return result


results = group_indices_by_step(df)
results


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os

# Function to load and plot an image with dynamic size adjustment
def plot_image_at_xy(ax, img_path, x, y, max_size=0.09):
    if os.path.exists(img_path):
        img = plt.imread(img_path)
        img_height, img_width = img.shape[:2]
        aspect_ratio = img_width / img_height

        # Calculate zoom based on max_size and aspect ratio
        trans = ax.transData
        fig = ax.get_figure()
        dpi = fig.dpi
        xlim = ax.get_xlim()
        data_width = xlim[1] - xlim[0]
        bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axes_width_pixels = bbox.width * dpi
        pixels_per_data = axes_width_pixels / data_width
        size_pixels = max_size * pixels_per_data
        zoom = size_pixels / max(img_width, img_height / aspect_ratio)

        imagebox = OffsetImage(img, zoom=zoom)
        ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0)
        ax.add_artist(ab)
        return ab
    else:
        print(f"Image {img_path} not found")
        return None


In [ ]:

# Create a figure and axis with increased height
fig, ax = plt.subplots(figsize=(24, 8.5))
map_color='jet'
# Assuming matrix and data are defined elsewhere
# Example placeholder for matrix and data
i_row=0
base_y = 0.35
y_step = 0.5
# Iterate through each bin (column of the matrix)

for indices in results:
    y_coord = base_y + (i_row * y_step)

    if len(indices) > 0:
        values = [df[i] for i in indices]
        num_ratemaps = len(indices)
        for i, (idx, value) in enumerate(zip(indices, values)):
            if sublayer:
                img_path = fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\control/{sublayer}/{map_color}/{idx}.png"
            else:

                img_path = fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\control/{map_color}/{idx}.png"

            plot_image_at_xy(ax, img_path, value, y_coord,  max_size=0.014)
    
    i_row += 1
# # Set plot limits with increased y-range
ax.set_xlim(0, 4.5)
ax.set_ylim(0, 8.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

# Set labels and title
ax.tick_params(axis='x', labelsize=20)
ax.set_xlabel("Information Content Rate (spikes / bit)", fontsize=24)
# Add vertical dashed lines at x = [1, 2, 3, 4]
ax.set_yticks([])
ax.vlines(x=[1, 2, 3, 4], ymin=0, ymax=8.5, colors='grey', linestyles='dashed', linewidth=3)
# Remove y-ticks since we're using bin labels
# Show plot
plt.tight_layout()
plt.show()

In [ ]:
import pynapple as nap
import numpy as np
import os 
os.chdir(r'Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp\src')
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap_ax,plot_path
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,Speed_filtered_spikes
map_color='jet'
base_nwb_folder = fr"S:\Sachuriga\nwb\test4neo"
df = exp_df.reset_index(drop=True)

sublayer = "deep"
if sublayer:
    df = exp_df[exp_df['sub_population']==sublayer].reset_index(drop=True)



for temp_name in np.unique(df['session_id']):
    print(temp_name)
    npdata = nap.load_file(fr"{base_nwb_folder }/{temp_name}")
    pos_cord = load_speed_fromNWB(npdata['XY_mid_brain'])

    raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(pos_cord[:,0], # times
                            pos_cord[:,1], # x
                            pos_cord[:,2], # y
                            filter_speed=True, 
                            min_speed = 0.05)
    
    

    temp_table = df[df['session_id']==temp_name]
    for i in range(len(temp_table)):

        spike_times = Speed_filtered_spikes(temp_table['spike_times'].iloc[i],
                            pos_cord[:,0],mask)
        
        maps = mapp.SpatialMap(box_size=[1.0, 1.0], bin_size=0.01, smoothing=0.1)
        rate_map = maps.rate_map(combined_array[:,1], combined_array[:,2], combined_array[:,0], spike_times)
        fig = plt.figure()
        ax = fig.add_subplot(111, xlim=[0, 1], ylim=[0, 1], aspect=1)
        ax.imshow(rate_map, interpolation='none', origin='upper',
                extent=(0, 1, 0, 1), vmin=0.01, cmap=fr'{map_color}')
        ax.grid(False)
                    # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        id=temp_table.index[i]
        ax.set_axis_off()
        if sublayer:
            fig.savefig(fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\exp/{sublayer}/{map_color}/{id}.png", bbox_inches='tight', pad_inches=0, transparent=True)
        else:
            fig.savefig(fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\exp/{map_color}/{id}.png", bbox_inches='tight', pad_inches=0, transparent=True)

import pandas as pd
import numpy as np

# 假设你的数据是一个Series
df =  df.reset_index(drop=True)['Information_content_rate']


def group_indices_by_step(df_col, step=0.075):
    used_indices = set()
    result = []
    values = df_col.copy()
    max_val = values.max()

    while len(used_indices) < len(values):
        remaining = values[~values.index.isin(used_indices)]
        if remaining.empty:
            break

        current_group = []
        current_val = remaining.min()
        current_idx = remaining.idxmin()
        current_group.append(current_idx)
        used_indices.add(current_idx)

        while True:
            target_val = current_val + step
            remaining = values[~values.index.isin(used_indices)]
            if remaining.empty:
                break

            # 找到大于等于 target_val 的值中最接近 target_val 的那个
            diffs = remaining - target_val
            diffs = diffs[diffs >= 0]
            if diffs.empty:
                break

            next_idx = diffs.idxmin()
            current_val = values[next_idx]
            current_group.append(next_idx)
            used_indices.add(next_idx)

        result.append(current_group)

    return result

results = group_indices_by_step(df)
results

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os

# Function to load and plot an image with dynamic size adjustment
def plot_image_at_xy(ax, img_path, x, y, max_size=0.09):
    if os.path.exists(img_path):
        img = plt.imread(img_path)
        img_height, img_width = img.shape[:2]
        aspect_ratio = img_width / img_height

        # Calculate zoom based on max_size and aspect ratio
        trans = ax.transData
        fig = ax.get_figure()
        dpi = fig.dpi
        xlim = ax.get_xlim()
        data_width = xlim[1] - xlim[0]
        bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axes_width_pixels = bbox.width * dpi
        pixels_per_data = axes_width_pixels / data_width
        size_pixels = max_size * pixels_per_data
        zoom = size_pixels / max(img_width, img_height / aspect_ratio)

        imagebox = OffsetImage(img, zoom=zoom)
        ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0)
        ax.add_artist(ab)
        return ab
    else:
        print(f"Image {img_path} not found")
        return None

# Create a figure and axis with increased height
fig, ax = plt.subplots(figsize=(24, 8.5))

# Assuming matrix and data are defined elsewhere
# Example placeholder for matrix and data
i_row=0
base_y = 0.35
y_step = 0.5
# Iterate through each bin (column of the matrix)

for indices in results:
    y_coord = base_y + (i_row * y_step)

    if len(indices) > 0:
        values = [df[i] for i in indices]
        num_ratemaps = len(indices)
        for i, (idx, value) in enumerate(zip(indices, values)):
            if sublayer:
                img_path = fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\exp\{sublayer}/{map_color}/{idx}.png"
            else:

                img_path = fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\exp\/{map_color}/{idx}.png"

            plot_image_at_xy(ax, img_path, value, y_coord,  max_size=0.014)
    i_row += 1

# # Set plot limits with increased y-range
ax.set_xlim(0, 4.5)
ax.set_ylim(0, 8.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)
# Set labels and title
ax.tick_params(axis='x', labelsize=20)
ax.set_xlabel("Information Content Rate (spikes / bit)", fontsize=24)
# Add vertical dashed lines at x = [1, 2, 3, 4]
ax.set_yticks([])
ax.vlines(x=[1, 2, 3, 4], ymin=0, ymax=8.5, colors='grey', linestyles='dashed', linewidth=3)
# Set labels and title

# Remove y-ticks since we're using bin labels
# Show plot
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Load Good units
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df_py = df_good[df_good['cell_type'] == "interneuron"]
df = df_py
base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
session = ["A", "B", "C", "Total"]

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# ... (Your existing code for loading data, filtering, and setting up the figure remains unchanged)

for session in session:
    # Filter for sessions
    if session == "Total":
        df_a = df
    else:
        df_a = df[df['session'] == session]

    # Separate into control and experimental groups
    control_df = df_a[df_a['animal_id'].isin(control_ids)]
    exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

    # Set Seaborn theme
    sns.set_theme(style="ticks")

    # Statistical comparisons for scalar metrics
    metrics = ['matlab_speedScores', 'Averate_rate']

    # Create figure with 2 rows and 6 columns
    fig, axes = plt.subplots(2, 6, figsize=(18, 10), width_ratios=[1, 0.5, 1, 0.5, 1, 0.5])
    axes = axes.flatten()  # Flatten for easier iteration

    # Define custom colors
    control_color = "red"  # Dark blue for Control
    exp_color = "magenta"  # Light blue for Experimental

    for idx, metric in enumerate(metrics):
        control_values = control_df[metric].dropna()
        exp_values = exp_df[metric].dropna()
        
        if len(control_values) > 0 and len(exp_values) > 0:
            control_mean = control_values.mean()
            exp_mean = exp_values.mean()
            control_sem = control_values.sem()
            exp_sem = exp_values.sem()
            
            print(f"\nComparison for {metric}:")
            print(f"Control mean: {control_mean:.2f} ± {control_sem:.2f}")
            print(f"Experimental mean: {exp_mean:.2f} ± {exp_sem:.2f}")
            
            # Mann-Whitney U test
            control_array = np.asarray(control_values.values, dtype=float)
            exp_array = np.asarray(exp_values.values, dtype=float)

            # Remove NaN values
            control_clean = control_array[~np.isnan(control_array)]
            exp_clean = exp_array[~np.isnan(exp_array)]

            # Perform Mann-Whitney U test
            u_stat, p_val = stats.mannwhitneyu(control_clean, exp_clean, alternative='two-sided')

            print(f"Mann-Whitney U statistic: {u_stat:.2f}, p-value: {p_val:.4f}")
            
            # Prepare data for Seaborn plotting
            plot_df = pd.DataFrame({
                'value': pd.concat([control_values, exp_values]),
                'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
            })
            
            # Check if deviation is "too large" (using coefficient of variation > 1 as threshold)
            all_values = plot_df['value']
            cv = all_values.std() / all_values.mean()  # Coefficient of variation
            use_log_scale = cv > 1 and all_values.min() > 0  # Ensure positive values for log scale
            
            # Filter out outliers (e.g., beyond 3 standard deviations)
            mean_val = all_values.mean()
            std_val = all_values.std()
            plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                       (plot_df['value'] <= mean_val + 3 * std_val)]
            
            # Boxplot (left subplot for each metric)
            box_ax = axes[idx * 2]  # Left column for boxplot
            sns.boxplot(
                data=plot_df_filtered,
                x='group',
                y='value',
                ax=box_ax,
                palette={"Control": control_color, "Experimental": exp_color},
                width=0.5
            )
            
            # Add individual points with matching colors
            sns.stripplot(
                data=plot_df_filtered,
                x='group',
                y='value',
                ax=box_ax,
                size=4,
                hue='group',
                palette={"Control": "black", "Experimental": "black"},
                alpha=0.4,
                jitter=0.1,
                legend=False
            )
            
            # Set title and labels for boxplot
            box_ax.set_title(f'{metric} Comparison')
            box_ax.set_ylabel(metric)
            box_ax.set_xlabel('Group')
            box_ax.yaxis.grid(False)
            box_ax.set(xlabel="")
            
            # Add p-value at the top of the boxplot
            box_ax.text(0.5, 0.95, f'p = {p_val:.4f}', 
                        horizontalalignment='center', 
                        verticalalignment='top', 
                        transform=box_ax.transAxes, 
                        fontsize=10)
        
            
            # Remove top and right spines
            box_ax.spines['top'].set_visible(False)
            box_ax.spines['right'].set_visible(False)
            box_ax.spines['bottom'].set_visible(True)
            box_ax.spines['left'].set_visible(True)

            # KDE plot (right subplot for each metric) as probability density
            kde_ax = axes[idx * 2 + 1]  # Right column for KDE
            sns.kdeplot(
                data=plot_df_filtered,
                y='value',
                hue='group',
                ax=kde_ax,
                palette={"Control": control_color, "Experimental": exp_color},
                fill=True,
                alpha=0.3,
                common_norm=False,  # Each group's KDE integrates to 1
                legend=False
            )
            
            # Match y-axis limits with boxplot
            kde_ax.set_ylim(box_ax.get_ylim())
            kde_ax.set_xticks([])  # Remove x-axis ticks
            kde_ax.set_yticks([])  # Remove y-axis ticks
            kde_ax.set_xlabel('')
            kde_ax.set_ylabel('')
            
            # Remove spines for KDE plot
            kde_ax.spines['top'].set_visible(False)
            kde_ax.spines['right'].set_visible(False)
            kde_ax.spines['bottom'].set_visible(False)
            kde_ax.spines['left'].set_visible(False)

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Load Good units
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
len(df_loaded)
df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df_py = df_good[df_loaded['buzaki_py_cell_type']=='pyramidal']
df = df_py

base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"

control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A", "B", "C", "Total"]

# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()
spi = 1.69
for idx, session in enumerate(sessions):
    # Filter for sessions
    if session == "Total":
        df_a = df
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['h_0_place_cell'] == 1) & 
                                      (control_df['Information_content_rate'] >= spi) & 
                                      (control_df['matlab_maxfsize'] >= 20) ]
        exp_df_place = exp_df[(exp_df['h_0_place_cell'] == 1) & 
                              (exp_df['Information_content_rate'] >= spi) & 
                              (exp_df['matlab_maxfsize'] >= 20) ]
    else:
        df_a = df[df['session'] == session]
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['h_0_place_cell'] == 1) & 
                                      (control_df['Information_content_rate'] >= spi) & 
                                      (control_df['matlab_maxfsize'] >= 20) ]
        exp_df_place = exp_df[(exp_df['h_0_place_cell'] == 1) & 
                              (exp_df['Information_content_rate'] >= spi) & 
                              (exp_df['matlab_maxfsize'] >= 20) ]

    # Calculate numbers for pie charts
    control_place = len(control_df_place)
    control_non_place = len(control_df) - control_place
    exp_place = len(exp_df_place)
    exp_non_place = len(exp_df) - exp_place

    # Z-Test for Two Proportions
    control_successes = control_place  # Number of place cells in control
    control_trials = len(control_df)   # Total pyramidal cells in control
    exp_successes = exp_place          # Number of place cells in experimental
    exp_trials = len(exp_df)           # Total pyramidal cells in experimental

    # Proportions
    p1 = control_successes / control_trials if control_trials > 0 else 0
    p2 = exp_successes / exp_trials if exp_trials > 0 else 0
    p_pooled = (control_successes + exp_successes) / (control_trials + exp_trials) if (control_trials + exp_trials) > 0 else 0

    from scipy.stats import binomtest
    result = binomtest(exp_place, len(exp_df), p=(control_place /len(control_df)), alternative='less')
    p_value = result.pvalue
    # Data for pie charts
    control_data = [control_place, control_non_place]
    exp_data = [exp_place, exp_non_place]
    labels = ['Place Cells', 'Non-Place Cells']
    colors = ['blue', 'lightblue']
    colors1 = ['cyan', 'lightcyan']

    # Create pie charts
    ax = axes[idx]
    ax.pie(control_data, labels=labels, colors=colors, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(-1, 0))
    ax.pie(exp_data, labels=labels, colors=colors1, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(1, 0))
    
    # Add title and p-value
    ax.set_title(f'Session {session}\np-value: {p_value:.4f}', pad=20)
    
    # Add group labels
    ax.text(-1, 1.2, 'Control', ha='center')
    ax.text(1, 1.2, 'Experimental', ha='center')

#fig.savefig(fr'{base_folder}/Functional_number_of_placecells.eps', format='eps', bbox_inches='tight')
#fig.savefig(fr'{base_folder}/Functional_number_of_placecells.png', format='png', bbox_inches='tight')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Assuming df_loaded is already defined
# Load Good units
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/good_units_with_tsnLabels.pkl')
len(df_loaded)
df_good = df_loaded[df_loaded['unit_quality']=="good"]
df_py = df_good[df_good['cell_type']=="interneuron"]
df=df_py

base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A", "B", "C", "Total"]

# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for idx, session in enumerate(sessions):
    # Filter for sessions
    if session == "Total":
        df_a = df

        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['matlab_speedScores'] >= 0.3)]
        exp_df_place = exp_df[(exp_df['matlab_speedScores'] >= 0.3)]
    else:
        df_a = df[df['session'] == session]
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['matlab_speedScores'] >= 0.3)]
        exp_df_place = exp_df[(exp_df['matlab_speedScores'] >= 0.3)]

    # Calculate numbers for pie charts
    control_place = len(control_df_place)
    control_non_place = len(control_df) - control_place
    exp_place = len(exp_df_place)
    exp_non_place = len(exp_df) - exp_place

    # Binomial test
    control_successes = control_place
    control_trials = len(control_df)
    exp_successes = exp_place
    exp_trials = len(exp_df)

    # Control proportion as null hypothesis
    p0 = control_successes / control_trials if control_trials > 0 else 0
    result = stats.binomtest(exp_successes, exp_trials, p=p0, alternative='two-sided')
    p_value = result.pvalue

    # Data for pie charts
    control_data = [control_place, control_non_place]
    exp_data = [exp_place, exp_non_place]
    labels = ['Speend Cells', 'Non-Speed Cells']
    colors = ['red', 'lightcoral']  # instead of 'red', 'lightred'
    colors1 = ['magenta', 'lightpink']  # instead of 'magenta', 'lightmagenta'

    # Create pie charts
    ax = axes[idx]
    ax.pie(control_data, labels=labels, colors=colors, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(-1, 0))
    ax.pie(exp_data, labels=labels, colors=colors1, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(1, 0))
    
    # Add title and p-value
    ax.set_title(f'Session {session}\np-value: {p_value:.4f}', pad=20)
    
    # Add group labels
    ax.text(-1, 1.2, 'Control', ha='center')
    ax.text(1, 1.2, 'Experimental', ha='center')
fig.savefig(fr'{base_folder}/Functional_number_of_speedcells.eps', format='eps', bbox_inches='tight')
fig.savefig(fr'{base_folder}/Functional_number_of_speedcells.png', format='png', bbox_inches='tight')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Load Good units
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df = df_good
base_folder = r"Q:\sachuriga\CR_CA1_paper\Results\Cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A", "B", "C", "Total"]

# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for idx, session in enumerate(sessions):
    # Filter for sessions
    if session == "Total":
        df_a = df
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_py = control_df[control_df['cell_type'] == "pyramidal"]
        control_df_int = control_df[(control_df['cell_type'] == "interneuron")]
        exp_df_py = exp_df[exp_df['cell_type'] == "pyramidal"]
        exp_df_int = exp_df[(exp_df['cell_type'] == "interneuron")]
    else:
        df_a = df[df['session'] == session]
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_py = control_df[control_df['cell_type'] == "pyramidal"]
        control_df_int = control_df[(control_df['cell_type'] == "interneuron")]
        exp_df_py = exp_df[exp_df['cell_type'] == "pyramidal"]
        exp_df_int = exp_df[(exp_df['cell_type'] == "interneuron")]

    # Calculate numbers for pie charts
    control_py = len(control_df_py)
    control_int = len(control_df_int)
    exp_py = len(exp_df_py)
    exp_int = len(exp_df_int)

    # Z-Test for Two Proportions
    control_successes = control_py  # Number of pyramidal cells in control
    control_trials = control_py + control_int  # Total cells in control
    exp_successes = exp_py  # Number of pyramidal cells in experimental
    exp_trials = exp_py + exp_int  # Total cells in experimental

    # Proportions
    p1 = control_successes / control_trials if control_trials > 0 else 0
    p2 = exp_successes / exp_trials if exp_trials > 0 else 0
    p_pooled = (control_successes + exp_successes) / (control_trials + exp_trials) if (control_trials + exp_trials) > 0 else 0

    # Standard error
    if p_pooled == 0 or p_pooled == 1 or control_trials == 0 or exp_trials == 0:
        z_stat = 0
        p_value = 1.0  # No difference if proportions are 0 or 1, or no data
    else:
        se = np.sqrt(p_pooled * (1 - p_pooled) * (1/control_trials + 1/exp_trials))
        z_stat = (p1 - p2) / se if se != 0 else 0
        p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # Two-tailed test

    # Data for pie charts
    control_data = [control_successes, control_int]
    exp_data = [exp_successes, exp_int]

    labels = ['Pyramidal', 'Interneuron']
    colors = ['blue', 'red']
    colors1 = ['cyan', 'magenta']

    # Create pie charts
    ax = axes[idx]
    ax.pie(control_data, labels=labels, colors=colors, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(-1, 0))
    ax.pie(exp_data, labels=labels, colors=colors1, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(1, 0))
    
    # Add title and p-value
    ax.set_title(f'Session {session}\np-value: {p_value:.4f}', pad=20)
    
    # Add group labels
    ax.text(-1, 1.2, 'Control', ha='center')
    ax.text(1, 1.2, 'Experimental', ha='center')

fig.savefig(fr'{base_folder}/PyvsInt.eps', format='eps', bbox_inches='tight')
fig.savefig(fr'{base_folder}/PyvsInt.png', format='png', bbox_inches='tight')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Load Good units
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
len(df_loaded)
df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df_py = df_good[df_good['cell_type'] == "pyramidal"]
df = df_py

base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"

control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A"]

# Create a figure with subplots
fig, axes = plt.subplots(1, 1, figsize=(12, 8))
axes = axes
spi = 1.69
for idx, session in enumerate(sessions):
    # Filter for sessions
    if session == "Total":
        df_a = df
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['h_0_place_cell'] == 1) & 
                                      (control_df['Information_content_rate'] >= spi) & 
                                      (control_df['matlab_maxfsize'] >= 20) &
                                      (control_df['firing_range'] < 10)]
        exp_df_place = exp_df[(exp_df['h_0_place_cell'] == 1) & 
                              (exp_df['Information_content_rate'] >= spi) & 
                              (exp_df['matlab_maxfsize'] >= 20) &
                              (exp_df['firing_range'] < 10)]
    else:
        df_a = df[df['session'] == session]
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['h_0_place_cell'] == 1) & 
                                      (control_df['Information_content_rate'] >= spi) & 
                                      (control_df['matlab_maxfsize'] >= 20) &
                                      (control_df['firing_range'] < 10)]
        exp_df_place = exp_df[(exp_df['h_0_place_cell'] == 1) & 
                              (exp_df['Information_content_rate'] >= spi) & 
                              (exp_df['matlab_maxfsize'] >= 20) &
                              (exp_df['firing_range'] < 10)]

    # Calculate numbers for pie charts
    control_place = len(control_df_place)
    control_non_place = len(control_df) - control_place
    exp_place = len(exp_df_place)
    exp_non_place = len(exp_df) - exp_place

    # Z-Test for Two Proportions
    control_successes = control_place  # Number of place cells in control
    control_trials = len(control_df)   # Total pyramidal cells in control
    exp_successes = exp_place          # Number of place cells in experimental
    exp_trials = len(exp_df)           # Total pyramidal cells in experimental

    # Proportions
    p1 = control_successes / control_trials if control_trials > 0 else 0
    p2 = exp_successes / exp_trials if exp_trials > 0 else 0
    p_pooled = (control_successes + exp_successes) / (control_trials + exp_trials) if (control_trials + exp_trials) > 0 else 0

    # Standard error and Z-test
    if p_pooled == 0 or p_pooled == 1 or control_trials == 0 or exp_trials == 0:
        z_stat = 0
        p_value = 1.0  # No difference if proportions are 0 or 1, or no data
    else:
        se = np.sqrt(p_pooled * (1 - p_pooled) * (1/control_trials + 1/exp_trials))
        z_stat = (p1 - p2) / se if se != 0 else 0
        p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # Two-tailed test

    # Data for pie charts
    control_data = [control_place, control_non_place]
    exp_data = [exp_place, exp_non_place]
    labels = ['Place Cells', 'Non-Place Cells']
    colors = ['blue', 'lightblue']
    colors1 = ['cyan', 'lightcyan']

    # Create pie charts
    ax = axes
    ax.pie(control_data, labels=labels, colors=colors, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(-1, 0))
    ax.pie(exp_data, labels=labels, colors=colors1, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(1, 0))
    
    # Add title and p-value
    ax.set_title(f'Session {session}\np-value: {p_value:.4f}', pad=20)
    
    # Add group labels
    ax.text(-1, 1.2, 'Control', ha='center')
    ax.text(1, 1.2, 'Experimental', ha='center')

#fig.savefig(fr'{base_folder}/Functional_number_of_placecells.eps', format='eps', bbox_inches='tight')
#fig.savefig(fr'{base_folder}/Functional_number_of_placecells.png', format='png', bbox_inches='tight')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Assuming df_loaded is already defined
# Load Good units
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/good_units_with_tsnLabels.pkl')
len(df_loaded)
df_good = df_loaded[df_loaded['unit_quality']=="good"]
df_py = df_good[df_good['cell_type']=="interneuron"]
df=df_py

base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A"]

# Create a figure with subplots
fig, axes = plt.subplots(1, 1, figsize=(12, 8))
axes = axes

for idx, session in enumerate(sessions):
    # Filter for sessions
    if session == "Total":
        df_a = df

        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['matlab_speedScores'] >= 0.3)]
        exp_df_place = exp_df[(exp_df['matlab_speedScores'] >= 0.3)]
    else:
        df_a = df[df['session'] == session]
        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]
        control_df_place = control_df[(control_df['matlab_speedScores'] >= 0.3)]
        exp_df_place = exp_df[(exp_df['matlab_speedScores'] >= 0.3)]

    # Calculate numbers for pie charts
    control_place = len(control_df_place)
    control_non_place = len(control_df) - control_place
    exp_place = len(exp_df_place)
    exp_non_place = len(exp_df) - exp_place

    # Binomial test
    control_successes = control_place
    control_trials = len(control_df)
    exp_successes = exp_place
    exp_trials = len(exp_df)

    # Control proportion as null hypothesis
    p0 = control_successes / control_trials if control_trials > 0 else 0
    result = stats.binomtest(exp_successes, exp_trials, p=p0, alternative='two-sided')
    p_value = result.pvalue

    # Data for pie charts
    control_data = [control_place, control_non_place]
    exp_data = [exp_place, exp_non_place]
    labels = ['Speend Cells', 'Non-Speed Cells']
    colors = ['red', 'lightcoral']  # instead of 'red', 'lightred'
    colors1 = ['magenta', 'lightpink']  # instead of 'magenta', 'lightmagenta'

    # Create pie charts
    ax = axes
    ax.pie(control_data, labels=labels, colors=colors, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(-1, 0))
    ax.pie(exp_data, labels=labels, colors=colors1, autopct='%1.1f%%', 
           startangle=90, radius=0.8, center=(1, 0))
    
    # Add title and p-value
    ax.set_title(f'Session {session}\np-value: {p_value:.4f}', pad=20)
    
    # Add group labels
    ax.text(-1, 1.2, 'Control', ha='center')
    ax.text(1, 1.2, 'Experimental', ha='center')
fig.savefig(fr'{base_folder}/Functional_number_of_speedcells.eps', format='eps', bbox_inches='tight')
fig.savefig(fr'{base_folder}/Functional_number_of_speedcells.png', format='png', bbox_inches='tight')

plt.tight_layout()
plt.show()